In [ ]:
clear()

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures

from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV

ModuleNotFoundError: No module named 'numpy'

In [ ]:
data = pd.read_csv("pulse2022_puf_51.csv")
print( 'Data set size: ', len( data ) )

In [ ]:
data=data[data['ANYWORK']>=0]
data['age']=2023-data['TBIRTH_YEAR']
X=data[['age','RRACE','EEDUC','MS']] # age, race, education, marital status
yy=data['ANYWORK']
yy.describe()
plt.hist(yy)

In [ ]:
X.replace(-99, 1, inplace = True) # replace -99 with 1. 
X_reg=pd.get_dummies(X, columns=X.columns[1:], drop_first=True)

In [ ]:
reg_linear = LinearRegression().fit(X_reg, yy)
reg_linear.score(X_reg, yy)
y_pred_linear=reg_linear.predict(X_reg)
for i in range(len(y_pred_linear)):
    if y_pred_linear[i] <1.5:
        y_pred_linear[i] = 1
    else:
        y_pred_linear[i] = 2
        
        
Accuracy_linear=1-np.sum(np.abs(yy-y_pred_linear))/len(yy)
print(Accuracy_linear)

In [ ]:
reg_linear.coef_

# Explore the linear model

In [ ]:
# count the number of missing values in each column and record columns with many missing values

remove_list=[]
threshold=20000
for i in data.columns:
    a = sum( data[i] == -88 ) + sum( data[i] == -99 ) 
    print(a,data[i].name)
    if a>threshold:
        remove_list.append(data[i].name)  # if we have too many missing data in this column, add this column to remove_list

In [ ]:
remove_list

In [ ]:
X=data[data.columns.difference(remove_list)] # let X be columns outside remove_list
X.columns

In [ ]:
# fill the remaining missing values with 1
X.replace(-88, 1, inplace = True)
X.replace(-99, 1, inplace = True)
X.describe()

In [ ]:
# find out which ones should be dropped, look like categorical variables, etc

for i in X.columns:
    a = len(pd.unique(X[i]))    # pd.unique gives the list of unique values in a column.
    print(a,X[i].name)

In [ ]:
more_remove_list=['ACTVDUTY1','KIDBSTR_LT5Y','SPND_SRCRV1','WEEK','SCRAM'] # why these?
numerical_var_list=['HWEIGHT','TSPNDFOOD','TSPNDPRPD','PWEIGHT','age','TBIRTH_YEAR'] 
X=X[X.columns.difference(more_remove_list)]

In [ ]:
X.columns.difference(numerical_var_list)

In [ ]:
X_reg=pd.get_dummies(X, columns=X.columns.difference(numerical_var_list), drop_first=True)
X_reg.describe()

In [ ]:
reg_linear = LinearRegression().fit(X_reg, yy)
reg_linear.score(X_reg, yy)

In [ ]:
reg_linear.coef_

In [ ]:
plt.plot(reg_linear.coef_)

In [ ]:
# let's normalize the covariates so we can compare betas (in order to figure out which ones are important)
Xreg_scaled=X_reg.copy()
for i in Xreg_scaled.columns:
    Xreg_scaled[i] = (Xreg_scaled[i]-Xreg_scaled[i].min())/(Xreg_scaled[i].max()-Xreg_scaled[i].min())
Xreg_scaled.describe()

In [ ]:
reg_linear = LinearRegression().fit(Xreg_scaled, yy)
reg_linear.score(Xreg_scaled, yy)

In [ ]:
reg_linear.coef_

In [ ]:
plt.plot(reg_linear.coef_)

In [ ]:
Xreg_scaled.columns[abs(reg_linear.coef_)>0.1]

In [ ]:

# put beta coefficients in a data frame
df = pd.DataFrame({'Name': Xreg_scaled.columns,
        'beta': reg_linear.coef_,
        'abs_val': abs(reg_linear.coef_) })   # we add the absolute column because we rank betas by their magnitude
df

In [ ]:
df.sort_values(by='abs_val', ascending=False) # sort 

In [ ]:
df.nlargest(10, ['abs_val']) # another way to sort

In [ ]:
C=df.nlargest(20, ['abs_val'])

In [ ]:
C

# Should we regularize the models? Let's focus on one state (regression)

In [ ]:
data = pd.read_csv("pulse2022_puf_51.csv")
print( 'Data set size: ', len( data ) )
data=data[data['TSPNDFOOD']>=0]
data['age']=2023-data['TBIRTH_YEAR']
yy=data['TSPNDFOOD'] # let's predict the expenditure on food and groceries

In [ ]:
remove_list=[]
threshold=20000
for i in data.columns:
    a = sum( data[i] == -88 ) + sum( data[i] == -99 ) 
    # print(a,data[i].name)
    if a>threshold:
        remove_list.append(data[i].name)  # if we have too many missing data in this column, add this column to remove_list
        
X=data[data.columns.difference(remove_list)] # let X be columns outside remove_list

X.replace(-99, 1, inplace = True) # replace -99 with 1. 
X.replace(-88, 1, inplace = True) # replace -99 with 1. 

X.columns

In [ ]:
XX=X[X['EST_ST']==25] # Massachusetts
yy=yy[X['EST_ST']==25]


In [ ]:
for i in XX.columns:
    a = len(pd.unique(XX[i]))
    print(a,XX[i].name)

In [ ]:
XX.columns[XX.isna().any()].tolist() # list all the columns that have any na values. 

In [ ]:
remove_list=['EST_MSA', 'KIDBSTR_LT5Y','TSPNDFOOD','ACTVDUTY1','EST_ST','REGION',
             'SPND_SRCRV1','WEEK','TBIRTH_YEAR','SCRAM','HEARING','THHLD_NUMPER',
             'LIVQTRRV','REMEMBERING','SEEING','SETTING','THHLD_NUMADLT','UNDERSTAND','ABIRTH_YEAR']
XX=XX[XX.columns.difference(remove_list)]

In [ ]:
numerical_var_list=['HWEIGHT','PWEIGHT','TBIRTH_YEAR','TSPNDFOOD','TSPNDPRPD','age']
XX_reg=pd.get_dummies(XX, columns=XX.columns.difference(numerical_var_list), drop_first=True)
XX_reg.columns

In [ ]:
XX_reg.columns[XX_reg.isna().any()].tolist()

In [ ]:
# split data randomly
train_x, test_x, train_y, test_y = train_test_split(XX_reg, yy, test_size=0.4, random_state = 138)

In [ ]:
# normalize the data (both training and test)

Xtrain_scaled=train_x.copy()
for i in Xtrain_scaled.columns:
    Xtrain_scaled[i] = (Xtrain_scaled[i]-Xtrain_scaled[i].min())/(Xtrain_scaled[i].max()-Xtrain_scaled[i].min())

    
Xtest_scaled=test_x.copy()
for i in Xtest_scaled.columns:
    Xtest_scaled[i] = (Xtest_scaled[i]-Xtest_scaled[i].min())/(Xtest_scaled[i].max()-Xtest_scaled[i].min())

In [ ]:
Xtrain_scaled.columns[Xtrain_scaled.isna().any()].tolist()

In [ ]:
Xtest_scaled.columns[Xtest_scaled.isna().any()].tolist()

In [ ]:
# linear model
reg1 = LinearRegression().fit(Xtrain_scaled, train_y) # train a linear model
y_pred_linear=reg1.predict(Xtest_scaled) # predict using the linear model on test data
Accuracy_linear_full=np.sum(np.square(y_pred_linear-test_y))/len(test_y) # evaluate MSE on test data
Accuracy_linear_full

In [ ]:
# linear model with manual variable selection
df = pd.DataFrame({'Name': Xtrain_scaled.columns,
        'beta': reg1.coef_,
        'abs_val': abs(reg1.coef_) })   # we add the absolute column because we rank betas by their magnitude
C=df.nlargest(52, ['abs_val'])

ind_chosen=C['Name']

reg1 = LinearRegression().fit(Xtrain_scaled[ind_chosen], train_y)
y_pred_linear=reg1.predict(Xtest_scaled[ind_chosen])
Accuracy_linear_small=np.sum(np.square(y_pred_linear-test_y))/len(test_y)
Accuracy_linear_small

In [ ]:
# Lasso
Lasso_model=LassoCV(cv=5, random_state=0).fit(Xtrain_scaled, train_y) # train Lasso with cross validation
y_pred_lasso=Lasso_model.predict(Xtest_scaled)
Accuracy_linear_lasso=np.sum(np.square(y_pred_lasso-test_y))/len(test_y)
Accuracy_linear_lasso

In [ ]:
# linear model with LASSO variable selection
ind_lasso=Xtrain_scaled.columns[abs(Lasso_model.coef_)>0]
reg1 = LinearRegression().fit(Xtrain_scaled[ind_lasso], train_y)
y_pred_linear=reg1.predict(Xtest_scaled[ind_lasso])
Accuracy_linear_post_lasso=np.sum(np.square(y_pred_linear-test_y))/len(test_y)
Accuracy_linear_post_lasso

In [ ]:
# Ridge 
Ridge_model = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1]).fit(Xtrain_scaled, train_y) # train ridge with cross validation
y_pred_ridge=Ridge_model.predict(Xtest_scaled)
Accuracy_linear_ridge=np.sum(np.square(y_pred_ridge-test_y))/len(test_y)
Accuracy_linear_ridge

In [ ]:
# Elastic net
EN_model = ElasticNetCV(cv=5, random_state=0).fit(Xtrain_scaled, train_y) # train elastic net with cross validation
y_pred_EN=EN_model.predict(Xtest_scaled)
Accuracy_linear_EN=np.sum(np.square(y_pred_EN-test_y))/len(test_y)
Accuracy_linear_EN

In [ ]:
# put all together in one place
print(Accuracy_linear_full, Accuracy_linear_small, Accuracy_linear_lasso, Accuracy_linear_post_lasso, Accuracy_linear_ridge, Accuracy_linear_EN)